In [1]:
import tensorflow as tf
from osgeo import gdal, gdalconst
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [2]:
driver = gdal.GetDriverByName("HFA")
driver.Register()

6

In [3]:
# load files
needclass_file = r"G:\Paper\JL\GF-2_3000_GS_302.dat"
ds = gdal.Open(needclass_file)
# getting dimensions
cols = ds.RasterXSize
rows = ds.RasterYSize
bands = ds.RasterCount
print(cols, rows, bands)

301 301 4


In [4]:
# read feature files pixel by pixel
needclassdsbypixel = []
for i in range(0, rows, 1):
    for j in range(0, cols, 1):
        data = ds.ReadAsArray(j, i, 1, 1).reshape(4,)/10000
        needclassdsbypixel.append(data)

In [5]:
print( needclassdsbypixel[0],  needclassdsbypixel[0].shape)

[ 0.0465  0.0637  0.0484  0.3027] (4,)


In [6]:
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean)
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)        
# make placeholder
with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 4],name="x")
    y = tf.placeholder(tf.float32, [None, 5],name="y")
    # keep_prob define dropout parameter
    keep_prob = tf.placeholder(tf.float32) 
# neruon network framework
with tf.name_scope("hlayer1"):
    with tf.name_scope("w1"):
        w1 = tf.Variable(tf.truncated_normal([4,100], stddev=0.1))
        variable_summaries(w1)
    with tf.name_scope("b1"):
        b1 = tf.Variable(tf.zeros([100])+0.1)
        variable_summaries(b1)
    L1 = tf.nn.tanh(tf.matmul(x, w1)+b1)
    L1_dropout = tf.nn.dropout(L1, keep_prob)
with tf.name_scope("hlayer2"):
    with tf.name_scope("w2"):
        w2 = tf.Variable(tf.truncated_normal([100,100], stddev=0.1))
        variable_summaries(w2)
    with tf.name_scope("b2"):
        b2 = tf.Variable(tf.zeros([100])+0.1)
        variable_summaries(b2)
    L2 = tf.nn.relu(tf.matmul(L1_dropout, w2)+b2)
    L2_dropout = tf.nn.dropout(L2, keep_prob)
with tf.name_scope("sotfmax"):
    with tf.name_scope("w3"):
        w3 = tf.Variable(tf.truncated_normal([100,5], stddev=0.1))
        variable_summaries(w3)
    with tf.name_scope("b3"):
        b3 = tf.Variable(tf.zeros([5])+0.1)
        variable_summaries(b3)
    prediction = tf.nn.softmax(tf.matmul(L2_dropout, w3)+b3)
# initial variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess,r"G:\Paper\python\TensorFlow\test06\net_mlp_lr==le-3_100_kp==0.5--/my_net.ckpt")
    da_pre = sess.run(prediction, feed_dict= {x:needclassdsbypixel,keep_prob:1.0})
    print(da_pre)

INFO:tensorflow:Restoring parameters from G:\Paper\python\TensorFlow\test06\net_mlp_lr==le-3_100_kp==0.5--/my_net.ckpt
[[  3.47179685e-34   3.84495761e-05   7.21722758e-22   4.25678728e-11
    9.99961495e-01]
 [  4.12200496e-35   5.26833765e-06   1.77100601e-21   1.82080739e-11
    9.99994755e-01]
 [  8.22057839e-36   3.56233407e-08   2.11635573e-21   8.07170411e-13
    1.00000000e+00]
 ..., 
 [  4.17747223e-08   0.00000000e+00   1.00000000e+00   0.00000000e+00
    9.33258784e-16]
 [  1.90497867e-10   0.00000000e+00   1.00000000e+00   0.00000000e+00
    5.96894655e-17]
 [  3.39131015e-11   0.00000000e+00   1.00000000e+00   0.00000000e+00
    3.87664786e-19]]


In [7]:
class_result1 = np.argmax(da_pre, axis=-1)
class_result2 = class_result1.reshape(301, 301) 
class_result2

array([[4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       [4, 4, 4, ..., 4, 4, 4],
       ..., 
       [4, 4, 4, ..., 2, 2, 2],
       [4, 4, 4, ..., 2, 2, 2],
       [4, 4, 4, ..., 2, 2, 2]], dtype=int64)

In [20]:
outDataset = driver.Create(r"G:\Paper\JL\test_res\302_mlp_5_cl.img",
                          301, 301, 1, gdal.GDT_Int32)

In [21]:
outBand = outDataset.GetRasterBand(1)
outBand.WriteArray(class_result2,0, 0)
A = outBand.ReadAsArray(0, 0, 20, 20)
A

array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 2, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4,

In [22]:
outBand.FlushCache()
outBand.GetStatistics(0, 1)

[0.0, 4.0, 2.4770035650820685, 1.4224792331211429]

In [23]:
geoTransform = ds.GetGeoTransform()
outDataset.SetGeoTransform(geoTransform)
proj = ds.GetProjection()
outDataset.SetProjection(proj)

0

In [24]:
# building pyramids
gdal.SetConfigOption("HFA_USE_RRD", "YES")
outDataset.BuildOverviews(overviewlist=[2, 4, 8, 16, 32, 64, 128])

0

In [25]:
geotransform = outDataset.GetGeoTransform()
print(geotransform[0])
proj = outDataset.GetProjection()
print(proj)

543616.671
PROJCS["WGS_1984_UTM_Zone_50N",GEOGCS["GCS_WGS_1984",DATUM["WGS_1984",SPHEROID["WGS_84",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",117.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1]]
